In [46]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import scipy as sp
from pydataset import data
from env import get_db_url, user, password, host
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

import acquire as acq
import os
directory = os.getcwd()

## Exercises

### 1. Create a new file named model_evaluation.py or model_evaluation.ipynb for these exercises.
### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.
|               | pred dog (+)  | pred not dog (-)  |
|:------------  |-----------:|-----------:|
| actual dog (+)   |         46 |         7  |
| actual not dog (-)   |         13 |         34 |

#### - In the context of this problem, what is a false positive?
#### - In the context of this problem, what is a false negative?
#### - How would you describe this model?

In [2]:
Accuracy = (34 + 46) / (34 + 7 + 13 + 46)
Accuracy
# - In the context of this problem, what is a false positive? Answer-
# I will assume we are predicting to find dog.

# FP: false positive happens when predicted (+) and actual (-) occurs, which in this case = 13
# Type I error 

0.8

In [3]:
# - In the context of this problem, what is a false negative? Answer-
# FN: happens when predicted negative (-) and actul positive (+) occurs, which in this case = 7
# Type II error

In [4]:
# - How would you describe this model? Answer-
# TP: We predicted dog, was dog (46)
# TN: We predicted not-dog, was not-dog (34)
# FP: We predicted dog, was not-dog (13) - Type I Error
# FN: We predicted not-dog, was dog (7) - Type II Error
TP = 46
TN = 34
FP = 13
FN = 7

#with an accuracy of 0.8, i think the model did well.

#### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.
#### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.
#### Use the predictions dataset and pandas to help answer the following questions:

In [5]:
ducks = pd.read_csv('c3.csv')
ducks.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


#### - An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [6]:
# True Positive Results

TP_m1 = ducks[(ducks.actual == 'Defect') & (ducks.model1 == 'Defect')]
TP_m1 = TP_m1[['actual', 'model1']]
TP_m1 = TP_m1.shape[0]

TP_m2 = ducks[(ducks.actual == 'Defect') & (ducks.model2 == 'Defect')]
TP_m2 = TP_m2[['actual', 'model2']]
TP_m2 = TP_m2.shape[0]

TP_m3 = ducks[(ducks.actual == 'Defect') & (ducks.model3 == 'Defect')]
TP_m3 = TP_m3[['actual', 'model3']]
TP_m3 = TP_m3.shape[0]

In [7]:
# True Negatives

TN_m1 = ducks[(ducks.actual == 'No Defect') & (ducks.model1 == 'No Defect')]
TN_m1 = TN_m1[['actual', 'model1']]
TN_m1 = TN_m1.shape[0]

TN_m2 = ducks[(ducks.actual == 'No Defect') & (ducks.model2 == 'No Defect')]
TN_m2 = TN_m2[['actual', 'model2']]
TN_m2 = TN_m2.shape[0]

TN_m3 = ducks[(ducks.actual == 'No Defect') & (ducks.model3 == 'No Defect')]
TN_m3 = TN_m3[['actual', 'model3']]
TN_m3 = TN_m3.shape[0]

In [8]:
# False Positives

FP_m1 = ducks[(ducks.actual == 'No Defect') & (ducks.model1 == 'Defect')]
FP_m1 = FP_m1[['actual', 'model1']]
FP_m1 = FP_m1.shape[0]

FP_m2 = ducks[(ducks.actual == 'No Defect') & (ducks.model2 == 'Defect')]
FP_m2 = FP_m2[['actual', 'model2']]
FP_m2 = FP_m2.shape[0]

FP_m3 = ducks[(ducks.actual == 'No Defect') & (ducks.model3 == 'Defect')]
FP_m3 = FP_m3[['actual', 'model3']]
FP_m3 = FP_m3.shape[0]

In [9]:
# False Negative

FN_m1 = ducks[(ducks.actual == 'Defect') & (ducks.model1 == 'No Defect')]
FN_m1 = FN_m1[['actual', 'model1']]
FN_m1 = FN_m1.shape[0]

FN_m2 = ducks[(ducks.actual == 'Defect') & (ducks.model2 == 'No Defect')]
FN_m2 = FN_m2[['actual', 'model2']]
FN_m2 = FN_m2.shape[0]

FN_m3 = ducks[(ducks.actual == 'Defect') & (ducks.model3 == 'No Defect')]
FN_m3 = FN_m3[['actual', 'model3']]
FN_m3 = FN_m3.shape[0]

In [10]:
ducks['baseline'] = 'No Defect'

In [11]:
m1_recall = TP_m1 / (TP_m1 + FN_m1)
m2_recall = TP_m2 / (TP_m2 + FN_m2)
m3_recall = TP_m3 / (TP_m3 + FN_m3)

m1_recall, m2_recall, m3_recall

(0.5, 0.5625, 0.8125)

In [12]:
positive = 'Defect'

subset = ducks[ducks.actual == positive]
model_recall = (subset.actual == subset.model1).mean()
bl_recall = (subset.baseline == subset.actual).mean()

print('Model 1')
print(f'Model Recall: {model_recall:.2%}')
print(f'Baseline recall: {bl_recall:.2%}')

Model 1
Model Recall: 50.00%
Baseline recall: 0.00%


In [13]:
positive = 'Defect'

subset = ducks[ducks.actual == positive]
model_recall = (subset.actual == subset.model2).mean()
bl_recall = (subset.baseline == subset.actual).mean()

print('Model 2')
print(f'Model Recall: {model_recall:.2%}')
print(f'Baseline recall: {bl_recall:.2%}')

Model 2
Model Recall: 56.25%
Baseline recall: 0.00%


In [14]:
positive = 'Defect'

subset = ducks[ducks.actual == positive]
model_recall = (subset.actual == subset.model3).mean()
bl_recall = (subset.baseline == subset.actual).mean()

print('Model 3')
print(f'Model Recall: {model_recall:.2%}')
print(f'Baseline recall: {bl_recall:.2%}')

Model 3
Model Recall: 81.25%
Baseline recall: 0.00%


#### - Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [15]:
m1_prec = TP_m1 / (TP_m1 + FP_m1)
m2_prec = TP_m2 / (TP_m2 + FP_m2)
m3_prec = TP_m3 / (TP_m3 + FP_m3)

m1_prec, m2_prec, m3_prec

(0.8, 0.1, 0.13131313131313133)

#### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).
#### At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).
#### Several models have already been developed with the data, and you can find their results here.
#### Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:
#### a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [17]:
paws = pd.read_csv('gives_you_paws.csv')
paws.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [18]:
paws.actual.value_counts()

dog    3254
cat    1746
Name: actual, dtype: int64

In [19]:
paws['baseline'] = 'dog'
paws.head()

,actual,model1,model2,model3,model4,baseline
0,cat,cat,dog,cat,dog,dog
1,dog,dog,cat,cat,dog,dog
2,dog,cat,cat,cat,dog,dog
3,dog,dog,dog,cat,dog,dog
4,cat,cat,cat,dog,dog,dog


In [20]:
# accuracy
bl_acc = (paws.actual == paws.baseline).mean()
m1_acc = (paws.actual == paws.model1).mean()
m2_acc = (paws.actual == paws.model2).mean()
m3_acc = (paws.actual == paws.model3).mean()
m4_acc = (paws.actual == paws.model4).mean()

bl_acc, m1_acc, m2_acc, m3_acc, m4_acc

(0.6508, 0.8074, 0.6304, 0.5096, 0.7426)

In [21]:
models = paws.loc[:, 'model1':'baseline'].columns.tolist()

output = []
for model in models:
    output.append({
        'model': model,
        'accuracy': (paws[model] == paws.actual).mean()
    })
    
metrics = pd.DataFrame(output)
metrics = metrics.sort_values(by='accuracy', ascending=False)
metrics

,model,accuracy
0,model1,0.8074
3,model4,0.7426
4,baseline,0.6508
1,model2,0.6304
2,model3,0.5096


### **model1 and model4 are better than baseline**

#### b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?

In [22]:
positive = 'dog'

# accuracy -- overall hit rate
model1_accuracy = (paws.model1 == paws.actual).mean()
model2_accuracy = (paws.model2 == paws.actual).mean()
model3_accuracy = (paws.model3 == paws.actual).mean()
model4_accuracy = (paws.model4 == paws.actual).mean()
baseline_accuracy = (paws.baseline == paws.actual).mean()

# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset1 = paws[paws.model1 == positive]
model1_precision = (subset1.model1 == subset1.actual).mean()
subset2 = paws[paws.model2 == positive]
model2_precision = (subset2.model1 == subset2.actual).mean()
subset3 = paws[paws.model3 == positive]
model3_precision = (subset3.model3 == subset3.actual).mean()
subset4 = paws[paws.model4 == positive]
model4_precision = (subset4.model4 == subset4.actual).mean()
subset_bl = paws[paws.baseline == positive]
baseline_precision = (subset_bl.baseline == subset_bl.actual).mean()

# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset = paws[paws.actual == positive]
model1_recall = (subset.model1 == subset.actual).mean()
model2_recall = (subset.model2 == subset.actual).mean()
model3_recall = (subset.model3 == subset.actual).mean()
model4_recall = (subset.model4 == subset.actual).mean()
baseline_recall = (subset.baseline == subset.actual).mean()


print(f'   model1 accuracy: {model1_accuracy:.2%}')
print(f'   model2 accuracy: {model2_accuracy:.2%}')
print(f'   model3 accuracy: {model3_accuracy:.2%}')
print(f'   model4 accuracy: {model4_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')
print()
print(f'   model1 recall: {model1_recall:.2%}')
print(f'   model2 recall: {model2_recall:.2%}')
print(f'   model3 recall: {model3_recall:.2%}')
print(f'   model4 recall: {model4_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')
print()
print(f'model1 precision: {model1_precision:.2%}')
print(f'model2 precision: {model2_precision:.2%}')
print(f'model3 precision: {model3_precision:.2%}')
print(f'model4 precision: {model4_precision:.2%}')
print(f'baseline precision: {baseline_precision:.2%}')


   model1 accuracy: 80.74%
   model2 accuracy: 63.04%
   model3 accuracy: 50.96%
   model4 accuracy: 74.26%
baseline accuracy: 65.08%

   model1 recall: 80.33%
   model2 recall: 49.08%
   model3 recall: 50.86%
   model4 recall: 95.57%
baseline recall: 100.00%

model1 precision: 89.00%
model2 precision: 80.03%
model3 precision: 65.99%
model4 precision: 73.12%
baseline precision: 65.08%


### **model4**

#### c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?

In [23]:
positive = 'cat'

# accuracy -- overall hit rate
model1_accuracy = (paws.model1 == paws.actual).mean()
model2_accuracy = (paws.model2 == paws.actual).mean()
model3_accuracy = (paws.model3 == paws.actual).mean()
model4_accuracy = (paws.model4 == paws.actual).mean()
baseline_accuracy = (paws.baseline == paws.actual).mean()

# precision -- how good are our positive predictions?
# precision -- model performance | predicted positive
subset1 = paws[paws.model1 == positive]
model1_precision = (subset1.model1 == subset1.actual).mean()
subset2 = paws[paws.model2 == positive]
model2_precision = (subset2.model1 == subset2.actual).mean()
subset3 = paws[paws.model3 == positive]
model3_precision = (subset3.model3 == subset3.actual).mean()
subset4 = paws[paws.model4 == positive]
model4_precision = (subset4.model4 == subset4.actual).mean()
subset_bl = paws[paws.baseline == positive]
baseline_precision = (subset_bl.baseline == subset_bl.actual).mean()

# recall -- how good are we at detecting actual positives?
# recall -- model performance | actual positive
subset = paws[paws.actual == positive]
model1_recall = (subset.model1 == subset.actual).mean()
model2_recall = (subset.model2 == subset.actual).mean()
model3_recall = (subset.model3 == subset.actual).mean()
model4_recall = (subset.model4 == subset.actual).mean()
baseline_recall = (subset.baseline == subset.actual).mean()


print(f'   model1 accuracy: {model1_accuracy:.2%}')
print(f'   model2 accuracy: {model2_accuracy:.2%}')
print(f'   model3 accuracy: {model3_accuracy:.2%}')
print(f'   model4 accuracy: {model4_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')
print()
print(f'   model1 recall: {model1_recall:.2%}')
print(f'   model2 recall: {model2_recall:.2%}')
print(f'   model3 recall: {model3_recall:.2%}')
print(f'   model4 recall: {model4_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')
print()
print(f'model1 precision: {model1_precision:.2%}')
print(f'model2 precision: {model2_precision:.2%}')
print(f'model3 precision: {model3_precision:.2%}')
print(f'model4 precision: {model4_precision:.2%}')
print(f'baseline precision: {baseline_precision:.2%}')

   model1 accuracy: 80.74%
   model2 accuracy: 63.04%
   model3 accuracy: 50.96%
   model4 accuracy: 74.26%
baseline accuracy: 65.08%

   model1 recall: 81.50%
   model2 recall: 89.06%
   model3 recall: 51.15%
   model4 recall: 34.54%
baseline recall: 0.00%

model1 precision: 68.98%
model2 precision: 81.13%
model3 precision: 35.83%
model4 precision: 80.72%
baseline precision: nan%


### **model1**

#### 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

- sklearn.metrics.accuracy_score
- sklearn.metrics.precision_score
- sklearn.metrics.recall_score
- sklearn.metrics.classification_report

In [29]:
# sklearn.metrics.accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)

sklearn.metrics.accuracy_score(paws.actual, paws.model1)

0.8074

In [30]:
sklearn.metrics.precision_score(paws.actual, paws.model2, pos_label='cat')

0.4841220423412204

In [31]:
sklearn.metrics.precision_score(paws.actual, paws.model3, pos_label='cat')

0.358346709470305

In [32]:
sklearn.metrics.precision_score(paws.actual, paws.model4, pos_label='cat')

0.8072289156626506

In [33]:
sklearn.metrics.precision_score(paws.actual, paws.model1, pos_label='dog')

0.8900238338440586

In [34]:
sklearn.metrics.precision_score(paws.actual, paws.model2, pos_label='dog')

0.8931767337807607

In [35]:
sklearn.metrics.precision_score(paws.actual, paws.model3, pos_label='dog')

0.6598883572567783

In [36]:
sklearn.metrics.precision_score(paws.actual, paws.model4, pos_label='dog')

0.7312485304490948

In [38]:
# sklearn.metrics.recall_score(y_true, y_pred, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')

sklearn.metrics.recall_score(paws.actual, paws.model1, pos_label='cat')

0.8150057273768614

In [39]:
sklearn.metrics.recall_score(paws.actual, paws.model2, pos_label='cat')

0.8906071019473081

In [40]:
sklearn.metrics.recall_score(paws.actual, paws.model3, pos_label='cat')

0.5114547537227949

In [41]:
sklearn.metrics.recall_score(paws.actual, paws.model4, pos_label='cat')

0.34536082474226804

In [42]:
sklearn.metrics.recall_score(paws.actual, paws.model1, pos_label='dog')

0.803318992009834

In [43]:
sklearn.metrics.recall_score(paws.actual, paws.model2, pos_label='dog')

0.49078057775046097

In [44]:
sklearn.metrics.recall_score(paws.actual, paws.model3, pos_label='dog')

0.5086047940995697

In [45]:
sklearn.metrics.recall_score(paws.actual, paws.model4, pos_label='dog')

0.9557467732022127

In [47]:
# sklearn.metrics.classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')

target_names = paws.actual.unique()
x = sklearn.metrics.classification_report(paws.actual, paws.model1, target_names=target_names, output_dict=True)
pd.DataFrame(x)

,cat,dog,accuracy,macro avg,weighted avg
precision,0.689772,0.890024,0.8074,0.789898,0.820096
recall,0.815006,0.803319,0.8074,0.809162,0.807400
f1-score,0.747178,0.844452,0.8074,0.795815,0.810484
support,1746.000000,3254.000000,0.8074,5000.000000,5000.000000
